In [22]:
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import torch

In [16]:
# Конфигурация
MODEL_PATH = '/home/lastinm/PROJECTS/credit_cards_detection/train/YOLOv12/runs/detect/train2/weights/best.pt'  # Путь к модели
TEST_DATA_DIR = '/home/lastinm/PROJECTS/credit_cards_detection/dataset/yolo/test/images'  # Папка с тестовыми изображениями
DATASET_YAML = '/home/lastinm/PROJECTS/credit_cards_detection/dataset/yolo/data.yaml'
RESULTS_DIR = 'results'  # Папка для сохранения результатов
CONF_THRESH = 0.5  # Порог уверенности для детекции
IOU_THRESH = 0.5  # Порог IoU для NMS

In [23]:
def evaluate_yolov12():
    # 1. Создание папки для результатов
    os.makedirs(RESULTS_DIR, exist_ok=True)
    
    # 2. Проверка существования тестовых данных
    if not os.path.exists(TEST_DATA_DIR):
        raise FileNotFoundError(f"Директория с тестовыми изображениями не найдена: {TEST_DATA_DIR}")
    
    # 3. Загрузка модели
    if not os.path.exists(MODEL_PATH):
        raise FileNotFoundError(f"Файл модели не найден: {MODEL_PATH}")
    
    model = YOLO(MODEL_PATH)
    
    # 4. Получение списка изображений
    test_images = [os.path.join(TEST_DATA_DIR, f) for f in os.listdir(TEST_DATA_DIR) 
                  if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    
    if not test_images:
        raise ValueError(f"В директории {TEST_DATA_DIR} не найдены изображения (.jpg/.png)")
    
    # 5. Обработка изображений
    for img_path in test_images:
        try:
            pred = model.predict(img_path, conf=CONF_THRESH, iou=IOU_THRESH, verbose=False)
            if pred and len(pred) > 0:
                pred_img = pred[0].plot()
                output_path = os.path.join(RESULTS_DIR, os.path.basename(img_path))
                Image.fromarray(pred_img[..., ::-1]).save(output_path)
        except Exception as e:
            print(f"Ошибка при обработке {img_path}: {str(e)}")
    
    # 6. Оценка метрик (исправленная версия)
    if os.path.exists(DATASET_YAML):
        metrics = model.val(
            data=DATASET_YAML,
            batch=8,
            imgsz=640,
            conf=CONF_THRESH,
            iou=IOU_THRESH,
            device='cuda' if torch.cuda.is_available() else 'cpu'
        )
        
        # Новый способ получения метрик
        print("\nРезультаты оценки:")
        print(f"mAP@0.5: {metrics.box.map:.4f}")
        print(f"mAP@0.5-0.95: {metrics.box.map75:.4f}")

        # Визуализация кривых
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, 6))
        for i, (precision, recall) in enumerate(zip(metrics.box.p, metrics.box.r)):
            plt.plot(recall, precision, label=f'Class {i} ({model.names[i]})')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve')
        plt.legend()
        plt.savefig(os.path.join(RESULTS_DIR, 'PR_curve.png'))
        plt.close()
    
    # Для precision и recall используем средние значения по классам
    if hasattr(metrics.box, 'p'):
        print(f"Precision: {sum(metrics.box.p)/len(metrics.box.p):.4f}")
    if hasattr(metrics.box, 'r'):
        print(f"Recall: {sum(metrics.box.r)/len(metrics.box.r):.4f}")
        
        # Сохранение метрик
        with open(os.path.join(RESULTS_DIR, 'metrics.txt'), 'w') as f:
            f.write(f"mAP@0.5: {metrics.box.map:.4f}\n")
            f.write(f"mAP@0.5-0.95: {metrics.box.map75:.4f}\n")
            if hasattr(metrics.box, 'p'):
                f.write(f"Precision: {sum(metrics.box.p)/len(metrics.box.p):.4f}\n")
            if hasattr(metrics.box, 'r'):
                f.write(f"Recall: {sum(metrics.box.r)/len(metrics.box.r):.4f}\n")
    else:
        print(f"Файл {DATASET_YAML} не найден. Расчет метрик пропущен.")

# if __name__ == "__main__":
evaluate_yolov12()

Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 11875MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 6129.2±644.2 MB/s, size: 153.6 KB)


val: Scanning /home/lastinm/PROJECTS/credit_cards_detection/dataset/yolo/valid/labels.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:00<00:00, 12.14it/s]


                   all         50        140      0.998      0.987       0.99      0.929
            CardHolder         41         41          1          1      0.995      0.912
            CardNumber         50         50      0.993       0.96      0.979      0.946
           DateExpired         49         49          1          1      0.995       0.93
Speed: 1.5ms preprocess, 2.2ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/detect/val9

Результаты оценки:
mAP@0.5: 0.9293
mAP@0.5-0.95: 0.9898
Precision: 0.9976
Recall: 0.9867
